# Product Detection Training

## Split Dataset
First we need to prepare our dataset, splitting it in train, validation and test set.

In [1]:
!python3 /home/app/src/scripts/split_dataset.py

Create foler structure for test set
100%|███████████████████████████████████████| 2920/2920 [01:09<00:00, 41.81it/s]
Create foler structure for val set
100%|████████████████████████████████████████| 584/584 [00:04<00:00, 131.79it/s]
Create foler structure for train set
100%|███████████████████████████████████████| 8185/8185 [08:42<00:00, 15.68it/s]


Go to File>Preferences>Settings and set  Output: Text Line Limit value to 500

## Train model

We will use comet as a visualization tool for our training experiments
We need to install comet_ml python library and create an account on [Comet](https://www.comet.com/site/).
Then go to [Settings](https://www.comet.com/account-settings/apiKeys)  and copy your api key.

In [2]:
!pip install -q comet_ml  # 1. install
!export COMET_API_KEY=<Your API Key>  # 2. paste API key

We also need to create a configuration .yaml file with the following content

In [3]:
%cat /home/app/src/data/yolo_data_v1.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license
# SKU110K Dataset

path:   /home/app/src/data/images  # dataset root dir
train:  train/            # train images (relative to 'path')
val:    val/              # val images (relative to 'path') 
test:   test/             # test images (optional)

# Classes
names:
  0: product

In [4]:
!python3 /home/app/yolov5/train.py --img 640 --batch 16 --epochs 20 --data /home/app/src/data/yolo_data_v1.yaml --weights yolov5m.pt --project /home/app/src/experiments --name exp

Overriding model.yaml nc=80 with nc=1

                 from  n    params  module                                  arguments                     
  0                -1  1      1760  models.common.Conv                      [3, 16, 6, 2, 2]              
  1                -1  1      4672  models.common.Conv                      [16, 32, 3, 2]                
  2                -1  1      4800  models.common.C3                        [32, 32, 1]                   
  3                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  4                -1  2     29184  models.common.C3                        [64, 64, 2]                   
  5                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  6                -1  3    156928  models.common.C3                        [128, 128, 3]                 
  7                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]             

Metrics from Validation set
![Metrics from Validation set](../resources/exp1graphs.png)

# Product and Missing Product Training 

In order to train a model able to detect products and missing products we have labelled missing products class for some images from our original dataset. For this we have used a free online tool called Roboflow. 
We are now goingto donwload the new dataset from our workspace.

In [5]:
!pip install -q roboflow
from roboflow import Roboflow

In [6]:
rf = Roboflow(api_key="YOUR-API-KEY-HERE")
project = rf.workspace("final-project-object-detection-for-instore-inventory-management").project("empty-spaces-in-a-supermarket-hanger-1upsp")
dataset = project.version(26).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Empty-spaces-in-a-supermarket-hanger-26 in yolov5pytorch:: 100%|██████████| 4854/4854 [00:09<00:00, 502.83it/s] 


Let's move our downdoaled dataset to our new data folder

In [7]:
!mv Empty-spaces-in-a-supermarket-hanger-26/ /home/app/src/datav2/

In [8]:
!python3 /home/app/yolov5/train.py --img 640 --batch 16 --epochs 300 --data /home/app/src/datav2/data.yaml --weights yolov5m.pt --project /home/app/src/experiments --name exp2

Overriding model.yaml nc=80 with nc=4

                from  n    params  module                                  arguments                     
0                -1  1      5280  models.common.Conv                      [3, 48, 6, 2, 2]              
1                -1  1     41664  models.common.Conv                      [48, 96, 3, 2]                
2                -1  2     65280  models.common.C3                        [96, 96, 2]                   
3                -1  1    166272  models.common.Conv                      [96, 192, 3, 2]               
4                -1  4    444672  models.common.C3                        [192, 192, 4]                 
5                -1  1    664320  models.common.Conv                      [192, 384, 3, 2]              
6                -1  6   2512896  models.common.C3                        [384, 384, 6]                 
7                -1  1   2655744  models.common.Conv                      [384, 768, 3, 2]              
8              

After 150 epochs we have a mAP of 0.83 in the validation set

Metrics from Validation set
![Metrics from Validation set](../resources/exp2graphs.png)